Testing Deep Learning on zernike polynomials up to degree 4

All values for $Z_n$ to be between -1 and 1

In [5]:
import random
from random import randrange
import numpy as np
import pandas as pd
import tensorflow as tf
import keras
from keras import models
from keras import layers
from keras.layers import Dense, Dropout, Flatten
from keras import optimizers
from keras import backend as K
from keras.callbacks import CSVLogger
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
import opticspy
# Using forked opticspy - https://github.com/aambrose-unh/opticspy/tree/plot_additions

Using TensorFlow backend.


In [10]:
from keras.initializers import RandomUniform
from keras.layers import LeakyReLU
from keras import regularizers

In [6]:
parent_dir_linux = '/content/'
# for using on google colab

parent_dir = ''
# for using on local windows machine

In [7]:
def extract_test_plan(test_num,
            file_path='test_plan_generator.csv'):
    print(file_path)
    full_test_plan = pd.read_csv(file_path,header=0)
    test_plan = full_test_plan[full_test_plan['test_num'] == test_num]
    test_plan = test_plan.reset_index(drop=True)
    
    return test_plan

In [8]:
#Create directory to store info
def create_dir(testing,parent_dir,linux=False,name_append=''):
    import os
    os.chdir(parent_dir)
    if linux==False:  
        if os.path.isdir('.\\Test_{}{}'.format(testing,name_append)):
            os.chdir('.\\Test_{}{}'.format(testing,name_append))
        else:
            os.makedirs('.\\Test_{}{}'.format(testing,name_append))
            os.chdir('.\\Test_{}{}'.format(testing,name_append))
    else:
        if os.path.isdir('./Test_{}{}'.format(testing,name_append)):
            os.chdir('./Test_{}{}'.format(testing,name_append))
        else:
            os.makedirs('./Test_{}{}'.format(testing,name_append))
            os.chdir('./Test_{}{}'.format(testing,name_append))

In [9]:
# Load data from folder:
# Data is a set of random coefficients to apply to zernike polynomials through 4th order
# The input data samples are 4 columns, x,y,dx,dy
# The coefficients will be the targets/output

def create_data(test_plan):    
    
    train_samp_num = int(test_plan.loc[0,'train_samp_num'])
    test_samp_num = int(test_plan.loc[0,'test_samp_num'])
    
# Load data
    train_coeff = np.load('../train_coeff.npy')[:train_samp_num]
    
    train_input_full = np.load('../train_input_normalized.npy',allow_pickle=True)[:train_samp_num]
    test_input_full = np.load('../test_input_normalized.npy',allow_pickle=True)[:test_samp_num]
    test_coeff = np.load('../test_coeff.npy')[:test_samp_num]
    
    train_input = np.array([train_input_full[n] for n in range(train_samp_num)])
    test_input = np.array([test_input_full[n] for n in range(test_samp_num)])
    
    return train_coeff,train_input,test_coeff,test_input


In [11]:
def model_create(test_plan,sqr_grid_width=50):
    model = models.Sequential()
    input_length = sqr_grid_width**2

    for i in range(int(test_plan['layer_num'].count())):
        reg_rate = float(test_plan['kernel_regularizer'][i])
       
        if i == 0:
            if test_plan['activation'][i] == 'LeakyReLU':
                    model.add(layers.Dense(int(test_plan['num_nodes'][i]), input_shape=(input_length,4),
                                   kernel_regularizer=regularizers.l2(reg_rate)))
                    model.add(LeakyReLU())    
            else:
                try:
                    model.add(layers.Dense(int(test_plan['num_nodes'][i]), activation=test_plan['activation'][i],
                                input_shape=(input_length,4),
                                   kernel_regularizer=regularizers.l2(reg_rate))) 
                except:
                    print('Check the input_shape parameter. This is specified in a separate file used to create \
                          the data stored in .npy files')

        
        #Flatten the data to make compatible for output
        elif test_plan['layer_type'][i] == 'Flatten':
                model.add(Flatten())
        
        elif i == int(test_plan['layer_num'].count())-1:
            model.add(layers.Dense(int(test_plan['num_nodes'][i]),
                                   kernel_regularizer=regularizers.l2(reg_rate)))
                
        #Define typical layer
        elif test_plan['activation'][i] == 'LeakyReLU':
            model.add(layers.Dense(int(test_plan['num_nodes'][i]),
                                   kernel_regularizer=regularizers.l2(reg_rate)))
            model.add(LeakyReLU())
                      
        else:
            model.add(layers.Dense(int(test_plan['num_nodes'][i]), activation=test_plan['activation'][i],
                                   kernel_regularizer=regularizers.l2(reg_rate)))
            
    optimizer = getattr(optimizers,test_plan['optimizer'][0])
    lr=test_plan['lr'][0]
    
    model.compile(optimizer=optimizer(lr=lr), loss=test_plan['loss'][0], metrics=['mae'])
    
    return model


In [12]:
# Simple scaling normalization
def zernike_gen_ones(batch_size=16,sqr_grid_width=50):
    
    while True:

        train_coeff = np.transpose([[random.uniform(-1.,1.) for Z1 in range(batch_size)],
                                    [random.uniform(-1.,1.) for Z2 in range(batch_size)],
                                    [random.uniform(-1.,1.) for Z3 in range(batch_size)],
                                    [random.uniform(-1.,1.) for Z4 in range(batch_size)],
                                    [random.uniform(-1.,1.) for Z5 in range(batch_size)],
                                    [random.uniform(-1.,1.) for Z6 in range(batch_size)],
                                    [random.uniform(-1.,1.) for Z7 in range(batch_size)],
                                    [random.uniform(-1.,1.) for Z8 in range(batch_size)],
                                    [random.uniform(-1.,1.) for Z9 in range(batch_size)],
                                    [random.uniform(-1.,1.) for Z10 in range(batch_size)],
                                    [random.uniform(-1.,1.) for Z11 in range(batch_size)],
                                    [random.uniform(-1.,1.) for Z12 in range(batch_size)],
                                    [random.uniform(-1.,1.) for Z13 in range(batch_size)],
                                    [random.uniform(-1.,1.) for Z14 in range(batch_size)],
                                    [random.uniform(-1.,1.) for Z15 in range(batch_size)]])
        
#         Divide x and y by sqr_grid_width to normalize between 0 and 1
        x = np.array([np.array([[i for i in range(sqr_grid_width)] for x in range(sqr_grid_width)]).flatten()
                           for z in range(batch_size)])/float(sqr_grid_width)
        y = np.array([np.array([[y for i in range(sqr_grid_width)] for y in range(sqr_grid_width)]).flatten()
                           for z in range(batch_size)])/float(sqr_grid_width)


        # z_grid to be used to create training data for dx and dy to create the gradient data
        z_grid_train = np.array([np.array(opticspy.zernike.Coefficient(Z1=train_coeff[z][0],
                            Z2=train_coeff[z][1], Z3=train_coeff[z][2], Z4=train_coeff[z][3], Z5=train_coeff[z][4],
                            Z6=train_coeff[z][5], Z7=train_coeff[z][6], Z8=train_coeff[z][7], Z9=train_coeff[z][8],
                            Z10=train_coeff[z][9], Z11=train_coeff[z][10], Z12=train_coeff[z][11], Z13=train_coeff[z][12],
                            Z14=train_coeff[z][13], Z15=train_coeff[z][14]).zernikematrix(l=sqr_grid_width))
                            for z in range(batch_size)])


        gradient_train = [np.gradient(z_grid_train[i]) for i in range(batch_size)]
        
        dx_train = np.array([np.array([x for x in gradient_train[i][0]]).flatten() for i in range(batch_size)])
        dy_train = np.array([np.array([y for y in gradient_train[i][1]]).flatten() for i in range(batch_size)])

        train_input = np.array([np.transpose([x[i], y[i], dx_train[i], dy_train[i]])
                      for i in range(batch_size)])

        yield train_input,train_coeff

In [13]:
# Simple scaling normalization
def zernike_gen(batch_size=16,sqr_grid_width=50):
    
    while True:

        train_coeff = np.transpose([[random.uniform(-2.,3.) for Z1 in range(batch_size)],
                                    [random.uniform(-1.,1.) for Z2 in range(batch_size)],
                                    [random.uniform(-1.,1.) for Z3 in range(batch_size)],
                                    [random.uniform(-9.,5.) for Z4 in range(batch_size)],
                                    [random.uniform(-4.,2.) for Z5 in range(batch_size)],
                                    [random.uniform(-3.,3.2) for Z6 in range(batch_size)],
                                    [random.uniform(-1.,1.4) for Z7 in range(batch_size)],
                                    [random.uniform(-1.,1.) for Z8 in range(batch_size)],
                                    [random.uniform(-1.,1.) for Z9 in range(batch_size)],
                                    [random.uniform(-1.,1.) for Z10 in range(batch_size)],
                                    [random.uniform(-1.,1.) for Z11 in range(batch_size)],
                                    [random.uniform(-1.,1.) for Z12 in range(batch_size)],
                                    [random.uniform(-1.,1.) for Z13 in range(batch_size)],
                                    [random.uniform(-1.,1.) for Z14 in range(batch_size)],
                                    [random.uniform(-1.,1.) for Z15 in range(batch_size)]])
        
#         Divide x and y by sqr_grid_width to normalize between 0 and 1
        x = np.array([np.array([[i for i in range(sqr_grid_width)] for x in range(sqr_grid_width)]).flatten()
                           for z in range(batch_size)])/float(sqr_grid_width)
        y = np.array([np.array([[y for i in range(sqr_grid_width)] for y in range(sqr_grid_width)]).flatten()
                           for z in range(batch_size)])/float(sqr_grid_width)


        # z_grid to be used to create training data for dx and dy to create the gradient data
        z_grid_train = np.array([np.array(opticspy.zernike.Coefficient(Z1=train_coeff[z][0],
                            Z2=train_coeff[z][1], Z3=train_coeff[z][2], Z4=train_coeff[z][3], Z5=train_coeff[z][4],
                            Z6=train_coeff[z][5], Z7=train_coeff[z][6], Z8=train_coeff[z][7], Z9=train_coeff[z][8],
                            Z10=train_coeff[z][9], Z11=train_coeff[z][10], Z12=train_coeff[z][11], Z13=train_coeff[z][12],
                            Z14=train_coeff[z][13], Z15=train_coeff[z][14]).zernikematrix(l=sqr_grid_width))
                            for z in range(batch_size)])


        gradient_train = [np.gradient(z_grid_train[i]) for i in range(batch_size)]
        
        dx_train = np.array([np.array([x for x in gradient_train[i][0]]).flatten() for i in range(batch_size)])
        dy_train = np.array([np.array([y for y in gradient_train[i][1]]).flatten() for i in range(batch_size)])

        train_input = np.array([np.transpose([x[i], y[i], dx_train[i], dy_train[i]])
                      for i in range(batch_size)])

        yield train_input,train_coeff

In [14]:
def block_gen(min_blk=2,max_blk=10,sqr_grid_width=50):
    x_block_size = randrange(min_blk,max_blk+1) 
    x_block_start = randrange(sqr_grid_width-1-x_block_size) 
    x_block = np.array(range(x_block_start,x_block_start+x_block_size))/float(sqr_grid_width)

    y_block_size = randrange(min_blk,max_blk+1) 
    y_block_start = randrange(sqr_grid_width-1-y_block_size) 
    y_block = np.array(range(y_block_start,y_block_start+y_block_size))/float(sqr_grid_width)
    
    return x_block,y_block

y_mask = lambda y,batch,block: np.isin(y[batch][:,1],block)

x_mask = lambda x,batch,block: np.isin(x[batch][:,0],block)

def zernike_gen_block(batch_size=16,sqr_grid_width=50):
   
    while True:
        
        train_coeff = np.transpose([[random.uniform(-2.,3.) for Z1 in range(batch_size)],
                                    [random.uniform(-1.,1.) for Z2 in range(batch_size)],
                                    [random.uniform(-1.,1.) for Z3 in range(batch_size)],
                                    [random.uniform(-9.,5.) for Z4 in range(batch_size)],
                                    [random.uniform(-4.,2.) for Z5 in range(batch_size)],
                                    [random.uniform(-3.,3.2) for Z6 in range(batch_size)],
                                    [random.uniform(-1.,1.4) for Z7 in range(batch_size)],
                                    [random.uniform(-1.,1.) for Z8 in range(batch_size)],
                                    [random.uniform(-1.,1.) for Z9 in range(batch_size)],
                                    [random.uniform(-1.,1.) for Z10 in range(batch_size)],
                                    [random.uniform(-1.,1.) for Z11 in range(batch_size)],
                                    [random.uniform(-1.,1.) for Z12 in range(batch_size)],
                                    [random.uniform(-1.,1.) for Z13 in range(batch_size)],
                                    [random.uniform(-1.,1.) for Z14 in range(batch_size)],
                                    [random.uniform(-1.,1.) for Z15 in range(batch_size)]])


        x = np.array([np.array([[i for i in range(sqr_grid_width)] for x in range(sqr_grid_width)]).flatten()
                           for z in range(batch_size)])/float(sqr_grid_width)

        y = np.array([np.array([[y for i in range(sqr_grid_width)] for y in range(sqr_grid_width)]).flatten()
                           for z in range(batch_size)])/float(sqr_grid_width)


        # z_grid to be used to create training data for dx and dy to create the gradient data
        z_grid_train = np.array([np.array(opticspy.zernike.Coefficient(Z1=train_coeff[z][0],
                            Z2=train_coeff[z][1], Z3=train_coeff[z][2], Z4=train_coeff[z][3], Z5=train_coeff[z][4],
                            Z6=train_coeff[z][5], Z7=train_coeff[z][6], Z8=train_coeff[z][7], Z9=train_coeff[z][8],
                            Z10=train_coeff[z][9], Z11=train_coeff[z][10], Z12=train_coeff[z][11], Z13=train_coeff[z][12],
                            Z14=train_coeff[z][13], Z15=train_coeff[z][14]).zernikematrix(l=sqr_grid_width))
                            for z in range(batch_size)])


        gradient_train = [np.gradient(z_grid_train[i]) for i in range(batch_size)]

        dx_train = np.array([np.array([x for x in gradient_train[i][0]]).flatten() for i in range(batch_size)])
        dy_train = np.array([np.array([y for y in gradient_train[i][1]]).flatten() for i in range(batch_size)])

        train_input = np.array([np.transpose([x[i], y[i], dx_train[i], dy_train[i]])
                            for i in range(batch_size)])

    #     NOTE: every sample in batch has the same block removed with this set up
    #     Generate random block
        x_block,y_block = block_gen(min_blk=2,max_blk=10,sqr_grid_width=sqr_grid_width)

    #     Remove random block
        for i in range(batch_size):
            train_input[i][(y_mask(train_input,i,y_block)) & (x_mask(train_input,i,x_block))] = 0


        yield train_input,train_coeff

In [15]:
# Create random matrix to add to dx and dy
def add_noise(original,batch_size=1,mu=0,sigma=.01):
    noise = [[random.gauss(mu, sigma) for x in range(len(original[0]))] for i in range(batch_size)]
    mod_data = original + noise
    return np.array(mod_data)

def zernike_gen_noise(batch_size=16,sqr_grid_width=50):
    
    while True:
        
        train_coeff = np.transpose([[random.uniform(-2.,3.) for Z1 in range(batch_size)],
                                    [random.uniform(-1.,1.) for Z2 in range(batch_size)],
                                    [random.uniform(-1.,1.) for Z3 in range(batch_size)],
                                    [random.uniform(-9.,5.) for Z4 in range(batch_size)],
                                    [random.uniform(-4.,2.) for Z5 in range(batch_size)],
                                    [random.uniform(-3.,3.2) for Z6 in range(batch_size)],
                                    [random.uniform(-1.,1.4) for Z7 in range(batch_size)],
                                    [random.uniform(-1.,1.) for Z8 in range(batch_size)],
                                    [random.uniform(-1.,1.) for Z9 in range(batch_size)],
                                    [random.uniform(-1.,1.) for Z10 in range(batch_size)],
                                    [random.uniform(-1.,1.) for Z11 in range(batch_size)],
                                    [random.uniform(-1.,1.) for Z12 in range(batch_size)],
                                    [random.uniform(-1.,1.) for Z13 in range(batch_size)],
                                    [random.uniform(-1.,1.) for Z14 in range(batch_size)],
                                    [random.uniform(-1.,1.) for Z15 in range(batch_size)]])

        x = np.array([np.array([[i for i in range(sqr_grid_width)] for x in range(sqr_grid_width)]).flatten()
                           for z in range(batch_size)])/float(sqr_grid_width)
        y = np.array([np.array([[y for i in range(sqr_grid_width)] for y in range(sqr_grid_width)]).flatten()
                           for z in range(batch_size)])/float(sqr_grid_width)


        # z_grid to be used to create training data for dx and dy to create the gradient data
        z_grid_train = np.array([np.array(opticspy.zernike.Coefficient(Z1=train_coeff[z][0],
                            Z2=train_coeff[z][1], Z3=train_coeff[z][2], Z4=train_coeff[z][3], Z5=train_coeff[z][4],
                            Z6=train_coeff[z][5], Z7=train_coeff[z][6], Z8=train_coeff[z][7], Z9=train_coeff[z][8],
                            Z10=train_coeff[z][9], Z11=train_coeff[z][10], Z12=train_coeff[z][11], Z13=train_coeff[z][12],
                            Z14=train_coeff[z][13], Z15=train_coeff[z][14]).zernikematrix(l=sqr_grid_width))
                            for z in range(batch_size)])


        gradient_train = [np.gradient(z_grid_train[i]) for i in range(batch_size)]

        dx_train = np.array([np.array([x for x in gradient_train[i][0]]).flatten() for i in range(batch_size)])
        dx_noise = add_noise(dx_train,batch_size=batch_size)

        dy_train = np.array([np.array([y for y in gradient_train[i][1]]).flatten() for i in range(batch_size)])
        dy_noise = add_noise(dy_train,batch_size=batch_size)

        train_input = np.array([np.transpose([x[i], y[i], dx_noise[i], dy_noise[i]])
                      for i in range(batch_size)])

        yield train_input,train_coeff

In [16]:
# Create random matrix to add to dx and dy and block region
def zernike_gen_blocknoise(batch_size=16,sqr_grid_width=50):
    
    while True:
        
        train_coeff = np.transpose([[random.uniform(-2.,3.) for Z1 in range(batch_size)],
                                    [random.uniform(-1.,1.) for Z2 in range(batch_size)],
                                    [random.uniform(-1.,1.) for Z3 in range(batch_size)],
                                    [random.uniform(-9.,5.) for Z4 in range(batch_size)],
                                    [random.uniform(-4.,2.) for Z5 in range(batch_size)],
                                    [random.uniform(-3.,3.2) for Z6 in range(batch_size)],
                                    [random.uniform(-1.,1.4) for Z7 in range(batch_size)],
                                    [random.uniform(-1.,1.) for Z8 in range(batch_size)],
                                    [random.uniform(-1.,1.) for Z9 in range(batch_size)],
                                    [random.uniform(-1.,1.) for Z10 in range(batch_size)],
                                    [random.uniform(-1.,1.) for Z11 in range(batch_size)],
                                    [random.uniform(-1.,1.) for Z12 in range(batch_size)],
                                    [random.uniform(-1.,1.) for Z13 in range(batch_size)],
                                    [random.uniform(-1.,1.) for Z14 in range(batch_size)],
                                    [random.uniform(-1.,1.) for Z15 in range(batch_size)]])

        x = np.array([np.array([[i for i in range(sqr_grid_width)] for x in range(sqr_grid_width)]).flatten()
                           for z in range(batch_size)])/float(sqr_grid_width)
        y = np.array([np.array([[y for i in range(sqr_grid_width)] for y in range(sqr_grid_width)]).flatten()
                           for z in range(batch_size)])/float(sqr_grid_width)


        # z_grid to be used to create training data for dx and dy to create the gradient data
        z_grid_train = np.array([np.array(opticspy.zernike.Coefficient(Z1=train_coeff[z][0],
                            Z2=train_coeff[z][1], Z3=train_coeff[z][2], Z4=train_coeff[z][3], Z5=train_coeff[z][4],
                            Z6=train_coeff[z][5], Z7=train_coeff[z][6], Z8=train_coeff[z][7], Z9=train_coeff[z][8],
                            Z10=train_coeff[z][9], Z11=train_coeff[z][10], Z12=train_coeff[z][11], Z13=train_coeff[z][12],
                            Z14=train_coeff[z][13], Z15=train_coeff[z][14]).zernikematrix(l=sqr_grid_width))
                            for z in range(batch_size)])


        gradient_train = [np.gradient(z_grid_train[i]) for i in range(batch_size)]

#     Add noise
        dx_train = np.array([np.array([x for x in gradient_train[i][0]]).flatten() for i in range(batch_size)])
        dx_noise = add_noise(dx_train,batch_size=batch_size)

        dy_train = np.array([np.array([y for y in gradient_train[i][1]]).flatten() for i in range(batch_size)])
        dy_noise = add_noise(dy_train,batch_size=batch_size)

        train_input = np.array([np.transpose([x[i], y[i], dx_noise[i], dy_noise[i]])
                      for i in range(batch_size)])
        
#     NOTE: every sample in batch has the same block removed with this set up?
#     Generate random block
        x_block,y_block = block_gen(min_blk=2,max_blk=10,sqr_grid_width=sqr_grid_width)

#     Remove random block
        for i in range(batch_size):
            train_input[i][(y_mask(train_input,i,y_block)) & (x_mask(train_input,i,x_block))] = 0

        yield train_input,train_coeff

In [17]:
def train_model(test_plan,model,test_num,gen=True,save_epochs=False,sqr_grid_width=50):
    
    epochs = int(test_plan['epochs'][0])
    steps_per_epoch = int(test_plan['steps_per_epoch'][0])
    batch_size = int(test_plan['batch_size'][0])

    #Display info for this test
    print("SETTINGS FOR THIS TEST:\n{} \n".format(test_plan))

    #log the info
    csv_logger = CSVLogger('Training.log')
    callbacks = [csv_logger]
    
    if save_epochs==True:
        checkpoint = ModelCheckpoint(filepath='epoch.{epoch:02d}.hdf5',
                                     monitor='val_loss',save_best_only=True,verbose=0, period=1) #
        callbacks.append(checkpoint)
    
    #Train the model 
    if gen == True:
        zern_gen = zernike_gen(batch_size=batch_size,sqr_grid_width=sqr_grid_width)
        val_gen = zernike_gen(batch_size=batch_size,sqr_grid_width=sqr_grid_width)

        history = model.fit_generator(generator=zern_gen,steps_per_epoch=steps_per_epoch,
                                      epochs=epochs, callbacks=callbacks, validation_data=val_gen,
                                      validation_steps=round(steps_per_epoch*.1)) 
    else:
        # Load data
        print("Loading Training data")
        train_coeff,train_input,test_coeff,test_input = create_data(test_plan)
        
        history = model.fit(train_input, train_coeff,callbacks=callbacks,
                    epochs=epochs, batch_size=batch_size,
                     validation_data=(test_input, test_coeff))
    
    return history

In [18]:
def train_model_noise(test_plan,model,test_num,gen=True,save_epochs=False,sqr_grid_width=50):
    
    epochs = int(test_plan['epochs'][0])
    steps_per_epoch = int(test_plan['steps_per_epoch'][0])
    batch_size = int(test_plan['batch_size'][0])
    # max_queue_size = int(test_plan['max_queue_size'][0])

    #Display info for this test
    print("SETTINGS FOR THIS TEST:\n{} \n".format(test_plan))

    #log the info
    csv_logger = CSVLogger('Training.log')
    
    callbacks = [csv_logger]
    
    if save_epochs==True:
        checkpoint = ModelCheckpoint(filepath='epoch.{epoch:02d}.hdf5',
                                     monitor='val_loss',save_best_only=True,verbose=0, period=1) #
        callbacks.append(checkpoint)
    
    #Train the model 
    if gen == True:
        zern_gen = zernike_gen_noise(batch_size=batch_size,sqr_grid_width=sqr_grid_width)
        val_gen = zernike_gen_noise(batch_size=batch_size,sqr_grid_width=sqr_grid_width)

        history = model.fit_generator(generator=zern_gen,steps_per_epoch=steps_per_epoch,
                                      epochs=epochs, callbacks=callbacks, validation_data=val_gen,
                                      validation_steps=round(steps_per_epoch*.1)) 
    else:
        # Load data
        print("Loading Training data")
        train_coeff,train_input,test_coeff,test_input = create_data(test_plan)
        
        history = model.fit(train_input, train_coeff,callbacks=callbacks,
                    epochs=epochs, batch_size=batch_size,
                     validation_data=(test_input, test_coeff))
    
    return history

In [19]:
def train_model_block(test_plan,model,test_num,gen=True,save_epochs=False,sqr_grid_width=50):
    
    epochs = int(test_plan['epochs'][0])
    steps_per_epoch = int(test_plan['steps_per_epoch'][0])
    batch_size = int(test_plan['batch_size'][0])
    # max_queue_size = int(test_plan['max_queue_size'][0])

    #Display info for this test
    print("SETTINGS FOR THIS TEST:\n{} \n".format(test_plan))

    #log the info
    csv_logger = CSVLogger('Training.log')
    
    callbacks = [csv_logger]
    
    if save_epochs==True:
        checkpoint = ModelCheckpoint(filepath='epoch.{epoch:02d}.hdf5',
                                     monitor='val_loss',save_best_only=True,verbose=0, period=1) #
        callbacks.append(checkpoint)
    
    #Train the model 
    if gen == True:
        zern_gen = zernike_gen_block(batch_size=batch_size,sqr_grid_width=sqr_grid_width)
        val_gen = zernike_gen_block(batch_size=batch_size,sqr_grid_width=sqr_grid_width)

        history = model.fit_generator(generator=zern_gen,steps_per_epoch=steps_per_epoch,
                                      epochs=epochs, callbacks=callbacks, validation_data=val_gen,
                                      validation_steps=round(steps_per_epoch*.1)) 
    else:
        # Load data
        print("Loading Training data")
        train_coeff,train_input,test_coeff,test_input = create_data(test_plan)
        
        history = model.fit(train_input, train_coeff,callbacks=callbacks,
                    epochs=epochs, batch_size=batch_size,
                     validation_data=(test_input, test_coeff))
    
    return history

In [20]:
def train_model_blocknoise(test_plan,model,test_num,gen=True,save_epochs=False,sqr_grid_width=50):
    
    epochs = int(test_plan['epochs'][0])
    steps_per_epoch = int(test_plan['steps_per_epoch'][0])
    batch_size = int(test_plan['batch_size'][0])
    # max_queue_size = int(test_plan['max_queue_size'][0])

    #Display info for this test
    print("SETTINGS FOR THIS TEST:\n{} \n".format(test_plan))

    #log the info
    csv_logger = CSVLogger('Training.log')
    
    callbacks = [csv_logger]
    
    if save_epochs==True:
        checkpoint = ModelCheckpoint(filepath='epoch.{epoch:02d}.hdf5',
                                     monitor='val_loss',save_best_only=True,verbose=0, period=1) #
        callbacks.append(checkpoint)
    
    #Train the model 
    if gen == True:
        zern_gen = zernike_gen_blocknoise(batch_size=batch_size,sqr_grid_width=sqr_grid_width)
        val_gen = zernike_gen_blocknoise(batch_size=batch_size,sqr_grid_width=sqr_grid_width)

        history = model.fit_generator(generator=zern_gen,steps_per_epoch=steps_per_epoch,
                                      epochs=epochs, callbacks=callbacks, validation_data=val_gen,
                                      validation_steps=round(steps_per_epoch*.1)) 
    else:
        # Load data
        print("Loading Training data")
        train_coeff,train_input,test_coeff,test_input = create_data(test_plan)
        
        history = model.fit(train_input, train_coeff,callbacks=callbacks,
                    epochs=epochs, batch_size=batch_size,
                     validation_data=(test_input, test_coeff))
    
    return history

In [21]:
def export_figs(history,testing):
    import matplotlib.pyplot as plt

    # Plot training & validation accuracy values
    plt.plot(history.history['mean_absolute_error'])
    plt.plot(history.history['val_mean_absolute_error'])
    plt.title('Model MAE - Test {}'.format(testing))
    plt.ylabel('MAE')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper left')
    plt.savefig('MAE_Plot_Test_{}.png'.format(testing), dpi=300, bbox_inches='tight')
    plt.clf()

    # Plot training & validation loss values
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model loss - Test {}'.format(testing))
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper left')
    plt.savefig('Loss_Plot_Test_{}.png'.format(testing), dpi=300, bbox_inches='tight')
    plt.clf()
    
    

In [22]:
def export_settings(test_plan,testing,history): 
    test_plan['best_val_mae'] = ""
    test_plan.loc[0,'best_val_mae'] = np.min(history.history['mean_absolute_error'])
    test_plan
    test_plan.to_csv('Test_{}_Settings.csv'.format(testing))
    
    
def export_test_data(model,test_num):
    model.save('model_test_{}.h5'.format(test_num))

In [23]:
def get_results(start,stop,parent_dir,sqr_grid_width=50,gen=True):
    for i in range(start,stop+1):
        print("\nBEGINNING TEST: {}".format(i))
        test_plan = extract_test_plan(test_num=i,file_path=parent_dir+'test_plan_generator.csv')

        print("Creating storage directory")
        create_dir(i,parent_dir,linux=False)
        
        print("Creating the model")
        model = model_create(test_plan,sqr_grid_width=sqr_grid_width)
        
        print(model.summary())
        
        print("Fitting model")
        history = train_model(test_plan,model,i,gen=gen,save_epochs=True,
                              sqr_grid_width=sqr_grid_width)
        
        print("Exporting plots")
        export_figs(history,i)

        print("Exporting CSV with test parameters and best mean absolute error result")
        export_settings(test_plan,i,history)
        
        print('Exporting Test Data')
        export_test_data(model,test_num=i)

In [24]:
# Added the load model option. Need to test.
def get_results_block(start,stop,parent_dir,sqr_grid_width=50,gen=True,load=False,load_name_append='',name_append=''):
    for i in range(start,stop+1):
        print("\nBEGINNING TEST: {}".format(i))
        test_plan = extract_test_plan(test_num=i,file_path=parent_dir+'test_plan_generator.csv')

        print("Creating storage directory")
        create_dir(i,parent_dir,linux=False,name_append=name_append)
        
        if load == False:
            print("Creating the model")
            model = model_create(test_plan,sqr_grid_width=sqr_grid_width)
        else:
            model = load_model('.\\Test_{}{}\\model_test_{}.h5'.format(i,load_name_append,i))
        
        print(model.summary())
        
        print("Fitting model")
        history = train_model_block(test_plan,model,i,gen=gen,save_epochs=True,
                              sqr_grid_width=sqr_grid_width)
        
        print("Exporting plots")
        export_figs(history,i)

        print("Exporting CSV with test parameters and best mean absolute error result")
        export_settings(test_plan,i,history)
        
        print('Exporting Test Data')
        export_test_data(model,test_num=i)

In [25]:
def get_results_noise(start,stop,parent_dir,sqr_grid_width=50,gen=True,load=False,load_name_append='',name_append=''):
    for i in range(start,stop+1):
        print("\nBEGINNING TEST: {}".format(i))
        test_plan = extract_test_plan(test_num=i,file_path=parent_dir+'test_plan_generator.csv')

        print("Creating storage directory")
        create_dir(i,parent_dir,linux=False,name_append=name_append)
        
        if load == False:
            print("Creating the model")
            model = model_create(test_plan,sqr_grid_width=sqr_grid_width)
        else:
            model = load_model('.\\Test_{}{}\\model_test_{}.h5'.format(i,load_name_append,i))
        
        print(model.summary())
        
        print("Fitting model")
        history = train_model_noise(test_plan,model,i,gen=gen,save_epochs=True,
                              sqr_grid_width=sqr_grid_width)
        
        print("Exporting plots")
        export_figs(history,i)

        print("Exporting CSV with test parameters and best mean absolute error result")
        export_settings(test_plan,i,history)
        
        print('Exporting Test Data')
        export_test_data(model,test_num=i)

In [27]:
def get_results_blocknoise(start,stop,parent_dir,sqr_grid_width=50,gen=True,load=False,
                           load_name_append='',name_append=''):
    for i in range(start,stop+1):
        print("\nBEGINNING TEST: {}".format(i))
        test_plan = extract_test_plan(test_num=i,file_path=parent_dir+'test_plan_generator.csv')

        print("Creating storage directory")
        create_dir(i,parent_dir,linux=False,name_append=name_append)
        
        if load == False:
            print("Creating the model")
            model = model_create(test_plan,sqr_grid_width=sqr_grid_width)
        else:
            model = models.load_model('..\\Test_{}{}\\model_test_{}.h5'.format(i,load_name_append,i))
        
        print(model.summary())
        
        print("Fitting model")
        history = train_model_blocknoise(test_plan,model,i,gen=gen,save_epochs=True,
                              sqr_grid_width=sqr_grid_width)
        
        print("Exporting plots")
        export_figs(history,i)

        print("Exporting CSV with test parameters and best mean absolute error result")
        export_settings(test_plan,i,history)
        
        print('Exporting Test Data')
        export_test_data(model,test_num=i)

In [65]:
# Using zernike_gen_ones -- zernike coefficients only ranged from -1 to 1
get_results(1,2,parent_dir,sqr_grid_width=50,gen=True)


BEGINNING TEST: 1
C:\Users\aambr\OneDrive\Documents\UNH Fall 2019\Research\zernike_polynomials\Data_and_Results_12-12-19\test_plan_generator.csv
Creating storage directory
Creating the model
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_45 (Dense)             (None, 2500, 32)          160       
_________________________________________________________________
dense_46 (Dense)             (None, 2500, 32)          1056      
_________________________________________________________________
dense_47 (Dense)             (None, 2500, 32)          1056      
_________________________________________________________________
flatten_12 (Flatten)         (None, 80000)             0         
_________________________________________________________________
dense_48 (Dense)             (None, 15)                1200015   
Total params: 1,202,287
Trainable params: 1,202,287
Non-trainable params: 0
______

Epoch 1/30
3200/3200 [==============================] - 150s 47ms/step - loss: 0.0303 - mean_absolute_error: 0.0709 - val_loss: 0.0247 - val_mean_absolute_error: 0.0574
Epoch 2/30
3200/3200 [==============================] - 148s 46ms/step - loss: 0.0242 - mean_absolute_error: 0.0563 - val_loss: 0.0229 - val_mean_absolute_error: 0.0562
Epoch 3/30
3200/3200 [==============================] - 149s 47ms/step - loss: 0.0238 - mean_absolute_error: 0.0544 - val_loss: 0.0234 - val_mean_absolute_error: 0.0542
Epoch 4/30
3200/3200 [==============================] - 148s 46ms/step - loss: 0.0233 - mean_absolute_error: 0.0530 - val_loss: 0.0234 - val_mean_absolute_error: 0.0505
Epoch 5/30
3200/3200 [==============================] - 149s 47ms/step - loss: 0.0234 - mean_absolute_error: 0.0524 - val_loss: 0.0231 - val_mean_absolute_error: 0.0511
Epoch 6/30
3200/3200 [==============================] - 149s 47ms/step - loss: 0.0232 - mean_absolute_error: 0.0512 - val_loss: 0.0229 - val_mean_absolute_

<Figure size 432x288 with 0 Axes>

In [67]:
get_results(1,12,parent_dir,sqr_grid_width=50,gen=True)


BEGINNING TEST: 1
C:\Users\aambr\OneDrive\Documents\UNH Fall 2019\Research\zernike_polynomials\Data_and_Results_12-12-19\test_plan_generator.csv
Creating storage directory
Creating the model
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_53 (Dense)             (None, 2500, 32)          160       
_________________________________________________________________
dense_54 (Dense)             (None, 2500, 32)          1056      
_________________________________________________________________
dense_55 (Dense)             (None, 2500, 32)          1056      
_________________________________________________________________
flatten_14 (Flatten)         (None, 80000)             0         
_________________________________________________________________
dense_56 (Dense)             (None, 15)                1200015   
Total params: 1,202,287
Trainable params: 1,202,287
Non-trainable params: 0
______

Epoch 1/30
3200/3200 [==============================] - 144s 45ms/step - loss: 0.1808 - mean_absolute_error: 0.1699 - val_loss: 0.1595 - val_mean_absolute_error: 0.1525
Epoch 2/30
3200/3200 [==============================] - 148s 46ms/step - loss: 0.1529 - mean_absolute_error: 0.1476 - val_loss: 0.1547 - val_mean_absolute_error: 0.1455
Epoch 3/30
3200/3200 [==============================] - 146s 46ms/step - loss: 0.1490 - mean_absolute_error: 0.1397 - val_loss: 0.1520 - val_mean_absolute_error: 0.1374
Epoch 4/30
3200/3200 [==============================] - 144s 45ms/step - loss: 0.1484 - mean_absolute_error: 0.1355 - val_loss: 0.1423 - val_mean_absolute_error: 0.1275
Epoch 5/30
3200/3200 [==============================] - 146s 46ms/step - loss: 0.1460 - mean_absolute_error: 0.1295 - val_loss: 0.1503 - val_mean_absolute_error: 0.1388
Epoch 6/30
3200/3200 [==============================] - 144s 45ms/step - loss: 0.1444 - mean_absolute_error: 0.1248 - val_loss: 0.1460 - val_mean_absolute_

3200/3200 [==============================] - 184s 57ms/step - loss: 0.1444 - mean_absolute_error: 0.1214 - val_loss: 0.1466 - val_mean_absolute_error: 0.1169
Epoch 8/30
3200/3200 [==============================] - 185s 58ms/step - loss: 0.1454 - mean_absolute_error: 0.1204 - val_loss: 0.1384 - val_mean_absolute_error: 0.1144
Epoch 9/30
3200/3200 [==============================] - 183s 57ms/step - loss: 0.1436 - mean_absolute_error: 0.1188 - val_loss: 0.1438 - val_mean_absolute_error: 0.1163
Epoch 10/30
3200/3200 [==============================] - 184s 57ms/step - loss: 0.1430 - mean_absolute_error: 0.1177 - val_loss: 0.1422 - val_mean_absolute_error: 0.1133
Epoch 11/30
3200/3200 [==============================] - 183s 57ms/step - loss: 0.1425 - mean_absolute_error: 0.1160 - val_loss: 0.1439 - val_mean_absolute_error: 0.1155
Epoch 12/30
3200/3200 [==============================] - 183s 57ms/step - loss: 0.1414 - mean_absolute_error: 0.1161 - val_loss: 0.1409 - val_mean_absolute_error: 0

3200/3200 [==============================] - 147s 46ms/step - loss: 0.1429 - mean_absolute_error: 0.1049 - val_loss: 0.1410 - val_mean_absolute_error: 0.1032
Epoch 10/30
3200/3200 [==============================] - 146s 46ms/step - loss: 0.1419 - mean_absolute_error: 0.1026 - val_loss: 0.1379 - val_mean_absolute_error: 0.1012
Epoch 11/30
3200/3200 [==============================] - 146s 46ms/step - loss: 0.1398 - mean_absolute_error: 0.1004 - val_loss: 0.1456 - val_mean_absolute_error: 0.1017
Epoch 12/30
3200/3200 [==============================] - 152s 47ms/step - loss: 0.1399 - mean_absolute_error: 0.0996 - val_loss: 0.1419 - val_mean_absolute_error: 0.0987
Epoch 13/30
3200/3200 [==============================] - 144s 45ms/step - loss: 0.1399 - mean_absolute_error: 0.0985 - val_loss: 0.1424 - val_mean_absolute_error: 0.0993
Epoch 14/30
3200/3200 [==============================] - 145s 45ms/step - loss: 0.1401 - mean_absolute_error: 0.0978 - val_loss: 0.1427 - val_mean_absolute_error:

3200/3200 [==============================] - 157s 49ms/step - loss: 0.1489 - mean_absolute_error: 0.1376 - val_loss: 0.1419 - val_mean_absolute_error: 0.1233
Epoch 4/30
3200/3200 [==============================] - 157s 49ms/step - loss: 0.1475 - mean_absolute_error: 0.1317 - val_loss: 0.1504 - val_mean_absolute_error: 0.1442
Epoch 5/30
3200/3200 [==============================] - 157s 49ms/step - loss: 0.1473 - mean_absolute_error: 0.1291 - val_loss: 0.1418 - val_mean_absolute_error: 0.1248
Epoch 6/30
3200/3200 [==============================] - 158s 49ms/step - loss: 0.1438 - mean_absolute_error: 0.1244 - val_loss: 0.1464 - val_mean_absolute_error: 0.1265
Epoch 7/30
3200/3200 [==============================] - 159s 50ms/step - loss: 0.1460 - mean_absolute_error: 0.1230 - val_loss: 0.1444 - val_mean_absolute_error: 0.1261
Epoch 8/30
3200/3200 [==============================] - 157s 49ms/step - loss: 0.1426 - mean_absolute_error: 0.1198 - val_loss: 0.1506 - val_mean_absolute_error: 0.11

3200/3200 [==============================] - 197s 62ms/step - loss: 0.7547 - mean_absolute_error: 0.5312 - val_loss: 0.7891 - val_mean_absolute_error: 0.5457
Epoch 10/30
3200/3200 [==============================] - 199s 62ms/step - loss: 0.7433 - mean_absolute_error: 0.5264 - val_loss: 0.7208 - val_mean_absolute_error: 0.5157
Epoch 11/30
3200/3200 [==============================] - 198s 62ms/step - loss: 0.7526 - mean_absolute_error: 0.5309 - val_loss: 0.7303 - val_mean_absolute_error: 0.5278
Epoch 12/30
3200/3200 [==============================] - 203s 63ms/step - loss: 0.7457 - mean_absolute_error: 0.5287 - val_loss: 0.8447 - val_mean_absolute_error: 0.5805
Epoch 13/30
3200/3200 [==============================] - 332s 104ms/step - loss: 0.7434 - mean_absolute_error: 0.5270 - val_loss: 0.8317 - val_mean_absolute_error: 0.5701
Epoch 14/30
3200/3200 [==============================] - 308s 96ms/step - loss: 0.7452 - mean_absolute_error: 0.5280 - val_loss: 0.7998 - val_mean_absolute_error

3200/3200 [==============================] - 261s 82ms/step - loss: 0.1424 - mean_absolute_error: 0.0983 - val_loss: 0.1396 - val_mean_absolute_error: 0.0957
Epoch 16/30
3200/3200 [==============================] - 260s 81ms/step - loss: 0.1428 - mean_absolute_error: 0.0978 - val_loss: 0.1390 - val_mean_absolute_error: 0.0948
Epoch 17/30
3200/3200 [==============================] - 265s 83ms/step - loss: 0.1418 - mean_absolute_error: 0.0976 - val_loss: 0.1453 - val_mean_absolute_error: 0.1049
Epoch 18/30
3200/3200 [==============================] - 265s 83ms/step - loss: 0.1422 - mean_absolute_error: 0.0976 - val_loss: 0.1394 - val_mean_absolute_error: 0.0955
Epoch 19/30
3200/3200 [==============================] - 265s 83ms/step - loss: 0.1416 - mean_absolute_error: 0.0970 - val_loss: 0.1414 - val_mean_absolute_error: 0.0944
Epoch 20/30
3200/3200 [==============================] - 265s 83ms/step - loss: 0.1411 - mean_absolute_error: 0.0965 - val_loss: 0.1427 - val_mean_absolute_error:

<Figure size 432x288 with 0 Axes>

In [68]:
get_results(13,14,parent_dir,sqr_grid_width=50,gen=True)


BEGINNING TEST: 13
C:\Users\aambr\OneDrive\Documents\UNH Fall 2019\Research\zernike_polynomials\Data_and_Results_12-12-19\test_plan_generator.csv
Creating storage directory
Creating the model
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_107 (Dense)            (None, 2500, 32)          160       
_________________________________________________________________
dense_108 (Dense)            (None, 2500, 32)          1056      
_________________________________________________________________
dense_109 (Dense)            (None, 2500, 32)          1056      
_________________________________________________________________
dense_110 (Dense)            (None, 2500, 32)          1056      
_________________________________________________________________
flatten_26 (Flatten)         (None, 80000)             0         
_________________________________________________________________
dense_111 (Dens

Epoch 1/30
3200/3200 [==============================] - 289s 90ms/step - loss: 0.7294 - mean_absolute_error: 0.4644 - val_loss: 0.1795 - val_mean_absolute_error: 0.2082
Epoch 2/30
3200/3200 [==============================] - 287s 90ms/step - loss: 0.1884 - mean_absolute_error: 0.1971 - val_loss: 0.1645 - val_mean_absolute_error: 0.1819
Epoch 3/30
3200/3200 [==============================] - 287s 90ms/step - loss: 0.1664 - mean_absolute_error: 0.1652 - val_loss: 0.1834 - val_mean_absolute_error: 0.1631
Epoch 4/30
3200/3200 [==============================] - 288s 90ms/step - loss: 0.1565 - mean_absolute_error: 0.1439 - val_loss: 0.1454 - val_mean_absolute_error: 0.1325
Epoch 5/30
3200/3200 [==============================] - 287s 90ms/step - loss: 0.1494 - mean_absolute_error: 0.1288 - val_loss: 0.1388 - val_mean_absolute_error: 0.1275
Epoch 6/30
3200/3200 [==============================] - 288s 90ms/step - loss: 0.1489 - mean_absolute_error: 0.1197 - val_loss: 0.1546 - val_mean_absolute_

<Figure size 432x288 with 0 Axes>

In [69]:
get_results(15,15,parent_dir,sqr_grid_width=50,gen=True)


BEGINNING TEST: 15
C:\Users\aambr\OneDrive\Documents\UNH Fall 2019\Research\zernike_polynomials\Data_and_Results_12-12-19\test_plan_generator.csv
Creating storage directory
Creating the model
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_116 (Dense)            (None, 2500, 32)          160       
_________________________________________________________________
dense_117 (Dense)            (None, 2500, 32)          1056      
_________________________________________________________________
dense_118 (Dense)            (None, 2500, 32)          1056      
_________________________________________________________________
flatten_28 (Flatten)         (None, 80000)             0         
_________________________________________________________________
dense_119 (Dense)            (None, 15)                1200015   
Total params: 1,202,287
Trainable params: 1,202,287
Non-trainable params: 0
_____

<Figure size 432x288 with 0 Axes>

In [28]:
get_results_blocknoise(6,6,parent_dir,sqr_grid_width=50,load=True,load_name_append='_Perfect',name_append='_BlockNoise')

W1214 14:31:34.442872 22832 deprecation_wrapper.py:119] From C:\Users\aambr\AppData\Local\Continuum\anaconda3\envs\zern_poly_2\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1214 14:31:34.455863 22832 deprecation_wrapper.py:119] From C:\Users\aambr\AppData\Local\Continuum\anaconda3\envs\zern_poly_2\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1214 14:31:34.526648 22832 deprecation_wrapper.py:119] From C:\Users\aambr\AppData\Local\Continuum\anaconda3\envs\zern_poly_2\lib\site-packages\keras\backend\tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W1214 14:31:34.527652 22832 deprecation_wrapper.py:119] From C:\Users\aambr\AppData\Local\Continuum\anaconda3\envs\zern_poly_2\lib\site-packages\keras\backend\tensorflow_back


BEGINNING TEST: 6
C:\Users\aambr\OneDrive\Documents\UNH Fall 2019\Research\zernike_polynomials\Data_and_Results_12-12-19\test_plan_generator.csv
Creating storage directory
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_75 (Dense)             (None, 2500, 32)          160       
_________________________________________________________________
dense_76 (Dense)             (None, 2500, 32)          1056      
_________________________________________________________________
dense_77 (Dense)             (None, 2500, 32)          1056      
_________________________________________________________________
flatten_19 (Flatten)         (None, 80000)             0         
_________________________________________________________________
dense_78 (Dense)             (None, 15)                1200015   
Total params: 1,202,287
Trainable params: 1,202,287
Non-trainable params: 0
_________________________

<Figure size 432x288 with 0 Axes>

In [30]:
# Trained from scratch
get_results_blocknoise(6,6,parent_dir,sqr_grid_width=50,load=False,name_append='_scratch_blocknoise')


BEGINNING TEST: 6
C:\Users\aambr\OneDrive\Documents\UNH Fall 2019\Research\zernike_polynomials\Data_and_Results_12-12-19\test_plan_generator.csv
Creating storage directory
Creating the model
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 2500, 32)          160       
_________________________________________________________________
dense_6 (Dense)              (None, 2500, 32)          1056      
_________________________________________________________________
dense_7 (Dense)              (None, 2500, 32)          1056      
_________________________________________________________________
flatten_2 (Flatten)          (None, 80000)             0         
_________________________________________________________________
dense_8 (Dense)              (None, 15)                1200015   
Total params: 1,202,287
Trainable params: 1,202,287
Non-trainable params: 0
______

<Figure size 432x288 with 0 Axes>

In [29]:
# Trained from scratch
get_results_block(6,6,parent_dir,sqr_grid_width=50,load=False,name_append='_scratch_block')


BEGINNING TEST: 6
C:\Users\aambr\OneDrive\Documents\UNH Fall 2019\Research\zernike_polynomials\Data_and_Results_12-12-19\test_plan_generator.csv
Creating storage directory
Creating the model
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 2500, 32)          160       
_________________________________________________________________
dense_2 (Dense)              (None, 2500, 32)          1056      
_________________________________________________________________
dense_3 (Dense)              (None, 2500, 32)          1056      
_________________________________________________________________
flatten_1 (Flatten)          (None, 80000)             0         
_________________________________________________________________
dense_4 (Dense)              (None, 15)                1200015   
Total params: 1,202,287
Trainable params: 1,202,287
Non-trainable params: 0
______

<Figure size 432x288 with 0 Axes>